In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import pickle
import sklearn

print(tf.__version__)

1.12.0


In [19]:
traindata = pd.read_pickle('data/traindata.pickle')
valdata   = pd.read_pickle('data/valdata.pickle')
testdata  = pd.read_pickle('data/testdata.pickle')

def numerize(item):
    if item == 'MWS':
        return 0
    elif item == 'HPL':
        return 1
    elif item == 'EAP':
        return 2
    raise ValueError('Author not recognized:', item)
    

Y_train = np.array([numerize(x) for x in list(traindata.author)])
Y_val = np.array([numerize(x) for x in list(valdata.author)])
Y_test = np.array([numerize(x) for x in list(testdata.author)])


In [17]:
Y_train[:5]

array([2, 1, 2, 0, 1])

In [3]:
# textual feature engineering
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words=STOPWORDS, max_features=100)

X_train = train_text_feats = vectorizer.fit_transform(traindata.text).todense()
X_val = val_text_feats = vectorizer.transform(valdata.text).todense() 
X_test = test_text_feats = vectorizer.transform(testdata.text).todense() 


In [4]:
# train_text_feats_df = pd.read_pickle('data/train_text_feats_df.pickle')
# val_text_feats_df   = pd.read_pickle('data/val_text_feats_df.pickle')
# test_text_feats_df  = pd.read_pickle('data/test_text_feats_df.pickle')

# for x in [train_text_feats_df, val_text_feats_df, test_text_feats_df]:
#     print(x.shape)

In [5]:

# Y_train_file = open('data/Y_train.pickle', 'wb')
# Y_val_file = open('data/Y_val.pickle', 'wb')
# Y_test_file = open('data/Y_test.pickle', 'wb')

# pickle.dump(Y_train, Y_train_file)
# pickle.dump(Y_val, Y_val_file)
# pickle.dump(Y_test, Y_test_file)

# for x in [Y_train_file, Y_val_file, Y_test_file]:
#     x.close()

In [6]:
model = tf.keras.Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(tf.keras.layers.Dense(64, activation='relu'))
# Add another:
model.add(tf.keras.layers.Dense(64, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [7]:

# Create a sigmoid layer:
tf.keras.layers.Dense(64, activation='sigmoid')
# Or:
tf.keras.layers.Dense(64, activation=tf.sigmoid)

# A linear layer with L1 regularization of factor 0.01 applied to the kernel matrix:
tf.keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))

# A linear layer with L2 regularization of factor 0.01 applied to the bias vector:
tf.keras.layers.Dense(64, bias_regularizer=tf.keras.regularizers.l2(0.01))

# A linear layer with a kernel initialized to a random orthogonal matrix:
tf.keras.layers.Dense(64, kernel_initializer='orthogonal')

# A linear layer with a bias vector initialized to 2.0s:
tf.keras.layers.Dense(64, bias_initializer=tf.keras.initializers.constant(2.0))

In [8]:
model = tf.keras.Sequential([
# Adds a densely-connected layer with 64 units to the model:
tf.keras.layers.Dense(64, activation='relu'),
# Add another:
tf.keras.layers.Dense(64, activation='relu'),
# Add a softmax layer with 10 output units:
tf.keras.layers.Dense(10, activation='softmax')])


In [35]:
# Configure a model for categorical classification.
model.compile(optimizer=tf.train.RMSPropOptimizer(0.01),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=[tf.keras.metrics.categorical_accuracy],
              n_classes=3
              )

In [13]:
type(Y_train)

list

In [36]:

model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_val, Y_val))

ValueError: Invalid argument "n_classes" passed to K.function with TensorFlow backend

In [26]:
# returns loss, accuracy
result = model.predict(X_val, verbose=2 )

In [32]:
result.shape

(1958, 10)

In [ ]:
import time
timenow = time.asctime( time.localtime(time.time()) )
print("Finished at: ", timenow)